In [1]:
#Mount drive

import os, sys

class FileSystem:
  def __init__(self, colab_dir="PersonTracking", local_dir="./", data_dir="data"): # replace with dlav path
    IN_COLAB = 'google.colab' in sys.modules
    if (IN_COLAB):
      from google.colab import drive
      drive.mount('/gdrive')
      self.root_dir = os.path.join("/gdrive/My Drive/", colab_dir)
    else:
      self.root_dir = local_dir
    self.data_dir = data_dir
    self.change_directory = False

  def data_path(self, name):
    return os.path.join(self.data_dir, name) if self.change_directory else os.path.join(self.root_dir, self.data_dir, name)

  def path(self, name):
    return os.path.join("./", name) if self.change_directory else os.path.join("./", self.root_dir, name)

  def cd(self):
    %cd {self.root_dir}
    %ls
    self.change_directory = True

fs = FileSystem()
fs.cd()

/Users/nodiz/Desktop/PersonTracking
Baseline.patch             __pycache__/
DetectedImages/            detLib/
Detection/                 detect3000.py
Frames/                    dr_utils.py
Input-Output/              output/
LICENSE.md                 pic/
MirkosREIDPlayground/      pippo3000.py
README.md                  reid.ipynb
Video/                     reid3000.py
VideoPersonTracking.ipynb  reidLib/
vids/              temp/
WeightsDetection/          tryYolo.ipynb
WeightsReid/


In [2]:
import glob
import random
import os
import sys
import math
import numpy as np
from PIL import Image
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import seaborn as sns 
sns.set()

In [5]:
# Build model

from reidLib.modeling.baseline import Baseline

__file__ = fs.path('reidLib/modeling')
sys.path.append(os.path.dirname(__file__))

def build_model(num_classes):
    #     model = Baseline(num_classes, cfg.MODEL.LAST_STRIDE, cfg.MODEL.PRETRAIN_PATH, cfg.MODEL.NECK, cfg.TEST.NECK_FEAT \ after)
    model = Baseline(num_classes, 1, "WeightsReid/resnet50-19c8e357.pth", 'bnneck', 'after', 'resnet50', 'imageNet')   # maybe try self instead of imageNet
    return model

device = torch.device('cpu')
model = build_model(1041)  # 1041 identities in training dataset
model.eval()  # evaluation mode
#model=torch.load('WeightsReid/resnet50_ibn_a_center_param_120.pth')
model.load_param('WeightsReid/WeightFab/center/resnet50_model_100.pth')
model.to(device);

AttributeError: module 'torch' has no attribute 'devie'

In [0]:
# Prepare tansform

normalize_transform = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = T.Compose([
    T.Resize([256, 128]),
    T.ToTensor(),
    normalize_transform
])


#Dataset  

class ImageDataset(Dataset):
    """Image Person ReID Dataset"""

    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path = self.dataset[index]
        img = Image.open(img_path).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)

        return img

# Dataloader

def val_collate_fn(batch):
    return torch.stack(batch, dim=0)



In [0]:
"""class Matcher():
    def __init__(self, num_query, feat_norm='yes'):
        self.num_query = num_query
        self.feat_norm = feat_norm

    def reset(self):
        self.feats = []
        self.pids = []

    def update(self, output, ids):
        feat, pid, camid = output
        self.feats.append(feat)
        self.pids.extend(np.asarray(ids))

    def compute(self):
        feats = torch.cat(self.feats, dim=0)
        if self.feat_norm == 'yes':
            print("The test feature is normalized")
            feats = torch.nn.functional.normalize(feats, dim=1, p=2)
        # query
        qf = feats[:self.num_query]
        q_pids = np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])
        # gallery
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])
        m, n = qf.shape[0], gf.shape[0]
        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        distmat.addmm_(1, -2, qf, gf.t())
        distmat = distmat.cpu().numpy()

        num_q, num_g = distmat.shape

        indices = np.argsort(distmat, axis=1)
        matches = (g_pids[indices] == q_pids[:, np.newaxis]).astype(np.int32)
"""

'class Matcher():\n    def __init__(self, num_query, feat_norm=\'yes\'):\n        self.num_query = num_query\n        self.feat_norm = feat_norm\n\n    def reset(self):\n        self.feats = []\n        self.pids = []\n        # self.camids = []\n\n    def update(self, output, ids):\n        feat, pid, camid = output\n        self.feats.append(feat)\n        self.pids.extend(np.asarray(ids))\n       # self.camids.extend(np.asarray(camid))\n\n    def compute(self):\n        feats = torch.cat(self.feats, dim=0)\n        if self.feat_norm == \'yes\':\n            print("The test feature is normalized")\n            feats = torch.nn.functional.normalize(feats, dim=1, p=2)\n        # query\n        qf = feats[:self.num_query]\n        q_pids = np.asarray(self.pids[:self.num_query])\n        q_camids = np.asarray(self.camids[:self.num_query])\n        # gallery\n        gf = feats[self.num_query:]\n        g_pids = np.asarray(self.pids[self.num_query:])\n        g_camids = np.asarray(self.ca

In [0]:
#Alahixandre
def reid(gallery_path='pic/*', query_path='pic/*'):
    
    #Load images query 
    imgs_query = sorted(glob.glob(os.path.join(query_path, "*.*"))) 
    num_query = np.shape(imgs_query)[0]  # first x images in dataset

    # Load images gallery
    imgs_gallery = sorted(glob.glob(os.path.join(gallery_path, "*.*"))) #pic = gallery_path

    imgs=imgs_query + imgs_gallery

    #print([os.path.basename(x) for x in imgs[:num_query]])
    #print([os.path.basename(x) for x in imgs[num_query:]])

    # build dataset and dataloader
    imdatas = ImageDataset(imgs, transform)
    demo_loader = DataLoader(
        imdatas, batch_size=np.shape(imgs)[0], shuffle=False, num_workers=4,
        collate_fn=val_collate_fn
    )


    # model evaluation

    with torch.no_grad():
      for batch in demo_loader:
        batch = batch.to(device)
        feat = model(batch) # (bs, 2048)
        feat_norm = 1
        if feat_norm:
          feat = torch.nn.functional.normalize(feat, dim=1, p=2)
        # query
        qf = feat[:num_query]

        # gallery
        gf = feat[num_query:]

        m, n = qf.shape[0], gf.shape[0]

        distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                  torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()

        distmat.addmm_(1, -2, qf, gf.t())
        distmat = distmat.cpu().numpy()

        return distmat,
    

In [0]:
distmat = reid(gallery_path='DetectedImages/MOT16-10-raw/*', query_path='YOLOv3-pedestrian/output/')
print(distmat)


2
['MOT16-11-raw2- pedestrian1.jpg', 'MOT16-11-raw3- pedestrian1.jpg']
['1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg', '1.jpg']
torch.Size([21, 2048])
2 19
[[0.78506577 0.76209617 0.79826295 0.7718278  0.82034135 0.86445737
  0.7821839  0.657313   0.7473388  0.79688156 0.64614594 0.74085414
  0.803856   0.7613851  0.80956984 0.7239249  0.7281227  0.7654996
  0.68638945]
 [0.94309735 0.9233998  0.88550365 0.925437   0.99413407 0.928411
  0.94571376 0.703187   0.88762677 0.95068026 0.80814004 0.7869024
  0.9521313  0.8593142  0.9758681  0.90749526 0.8401933  0.8944113
  0.8356309 ]]
